In [141]:
import json
import pandas
import os
import re
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
from  itertools import tee, chain, islice, groupby
from math import log
from operator import itemgetter,attrgetter
from itertools import takewhile
import requests
import re
from collections import namedtuple 

### Remarks
- 2018 has 990 judgements. 786 has justification. That means 20% of judgements has no justification. Quite weird. It turns out that half of these judgements contains word `oddalać (remove)` in some form. What about rest of judgements? Why they do not contain justification?   

In [142]:
root_dir = '/home/marcin/Desktop/SemestrVIII/PJN'
year = "2018"
json_data_dir = f"{root_dir}/data/json"

patterns = {
    re.compile('A?C.*') : 'civil',
    re.compile('A?U.*') : 'insurance',
    re.compile('A?K.*') : 'criminal', 
    re.compile('G.*') :  'economic',
    re.compile('A?P.*'): 'work', 
    re.compile('R.*'): 'family', 
    re.compile('W.*') : 'violations', 
    re.compile('Am.*'): 'competition'  ,
    re.compile('.*'): 'other' 
    
    }

common_words = ['w', 'z', 'na', 'i', 'do',
 'nie', 'o', 'k', 'r', 'że',
 'art', 'dnia', 'się', 'od', 'a',
 'przez', 'sąd', 'roku', 'pracy', 'za']

common_tagged_words =['w:prep', 'z:prep', 'na:prep', 
                      'i:conj', 'do:prep', 'nie:qub', 'dzień:subst',
                     'on:ppron3', 'o:prep', 'rok:brev', 'sąd:subst',
                     'że:comp', 'koło:brev', 'praca:subst', 'ten:adj',
                     'artykuł:brev', 'się:qub', 'od:prep', 'rok:subst', 'przez:prep']

In [143]:
def extract_file_number(filename):
    return int(filename.split('-')[1].split('.')[0])

def is_2017_in_file(filename):
    return 2716 <= extract_file_number(filename) <= 3163 

def is_2018_in_file(filename):
    return 3163 <= extract_file_number(filename) <=  3173

In [144]:
Category = namedtuple('Category',['label','caseNumber'])
Text = namedtuple('Text',['id','words'])
WithCategory = namedtuple('WithCategory', ['judgement','category'])
Line = namedtuple('Line', ['id','words','category'])

In [145]:
def clean_text(line):
    justification = line.split('<h2>UZASADNIENIE</h2>')[1]
    notags = re.sub(r"<[^>]*>", " ", justification)
    nobreaks = re.sub(r"-\n", " ", notags)
    nodigits = re.sub(r"\d+", " ", nobreaks)
    noromans = re.sub(r"\b[XVILMC]+\b", "", nodigits)
    lower = noromans.lower()
    words = filter(lambda x: x not in common_words, re.findall(r"\w+", lower))
    return words


def filter_judgements(judgement):
    return year in judgement['judgmentDate']  and \
    judgement['courtType'] in ['COMMON', 'SUPREME'] and \
    '<h2>UZASADNIENIE</h2>' in judgement['textContent']


def map_category(judgement):
    caseNumber = judgement['courtCases'][0]['caseNumber'].split(' ')
    _, label = next(
        filter(lambda pat: pat[0].match(caseNumber[1]), patterns.items()))
    return Category(label, caseNumber)


def judgements_raw(filename):
    with open(os.path.join(json_data_dir, filename), 'r') as jsonFile:
        judgements = json.load(jsonFile)['items']
    return judgements

In [146]:
def clean_tagged(line):
    _id, text = line
    words = filter(lambda word: word not in common_tagged_words,
                   text.split(' '))
    return Text(_id, words)


def tagged(categories_by_id):
    lines = map(lambda l: Text._make(l.split(';')),
                open('tagging-2018.data', 'r'))
    int_id = map(lambda l: (int(l[0]),l[1]),lines)
    filtered = filter(lambda l: l[0] in categories_by_id, int_id)
    cleaned = map(clean_tagged, filtered)
    return map(
        lambda text: Line(text.id, text.words, categories_by_id[text.id]),
        cleaned)

In [147]:
judgements_files = filter(is_2018_in_file, os.listdir(json_data_dir))
texts = chain.from_iterable(map(judgements_raw, judgements_files))
filtered = filter(filter_judgements, texts)
with_categories = map(lambda j: WithCategory(j, map_category(j).label), filtered)
cleaned = map(
    lambda j: Line(j.judgement['id'], clean_text(j.judgement['textContent']), j.category),
    with_categories)
cleaned_1, cleaned_2 = tee(cleaned)

In [148]:
judgements_files = filter(is_2018_in_file, os.listdir(json_data_dir))
texts = chain.from_iterable(map(judgements_raw, judgements_files))
filtered = filter(filter_judgements, texts)
with_categories = map(lambda j: WithCategory(j, map_category(j).label), filtered)
cleaned = map(
    lambda j: Line(j.judgement['id'], clean_text(j.judgement['textContent']), j.category),
    with_categories)
cleaned_1, cleaned_2 = tee(cleaned)

In [149]:
cat_id = dict(map(lambda line: (line.id, line.category), cleaned_2))
tagged_categories = tagged(cat_id)
tagged_sorted = sorted(tagged_categories,key= attrgetter('category'))
tagged_groups = groupby(tagged_sorted,attrgetter('category'))
tagged_grouped = dict((k, list(g)) for k, g in tagged_groups)

In [150]:
sorted_categories = sorted(cleaned_1, key=itemgetter(2))
groups = groupby(sorted_categories, itemgetter(2))
grouped = dict((k, list(g)) for k, g in groups)

In [166]:
next(tagged_grouped['civil'][0].words)

'wnieść:praet'